In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import urllib.request
import glob
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import math
import torch.nn.init as init

In [ ]:
class Config:
    # --- THAM SỐ BẮT BUỘC ---
    # !!! THAY ĐỔI ĐƯỜNG DẪN NÀY tới thư mục chứa file CSV của bạn
    DATA_ROOT = '../dataset'

    MODEL_NAME = 'mlp'  # Lựa chọn giữa 'mlp', 'custom1', 'custom2'
    EPOCHS = 15
    LEARNING_RATE = 1e-3
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    OUTPUT_PATH = 'save/best_model.pth'
    
    # Các tham số cho việc tải dữ liệu (để trống nếu không dùng)
    SOURCE_CSV = None
    DOWNLOAD_URL = None

args = Config()
print(f"Sử dụng thiết bị: {args.DEVICE}")

In [ ]:
class IoTID20CSVDataset(Dataset):
    def __init__(self, csv_file, feature_cols=None, label_col='label', scaler=None, label_encoder=None):
        df = pd.read_csv(csv_file)
        if feature_cols is None:
            feature_cols = [c for c in df.columns if c != label_col]
        self.feature_cols = feature_cols
        self.label_col = label_col
        X = df[feature_cols].values.astype(np.float32)
        y = df[label_col].values

        if scaler is None:
            scaler = StandardScaler()
            X = scaler.fit_transform(X)
        else:
            X = scaler.transform(X)
        self.scaler = scaler

        if label_encoder is None:
            label_encoder = LabelEncoder()
            y = label_encoder.fit_transform(y)
        else:
            y = label_encoder.transform(y)
        self.label_encoder = label_encoder

        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y.astype(np.int64))

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


def _standardize_and_split_source_csv(data_root, out_train, out_test, source_csv=None):
    candidates = []
    if source_csv and os.path.isfile(source_csv):
        candidates = [source_csv]
    else:
        candidates = [p for p in glob.glob(os.path.join(data_root, '*.csv')) if os.path.basename(p).lower() not in {'train.csv', 'test.csv'}]
    if not candidates:
        return False
    
    print(f"Đang xử lý file CSV nguồn: {candidates[0]}")
    df = pd.read_csv(candidates[0], skipinitialspace=True)
    df = df.drop_duplicates()
    for col in ['Flow_ID', 'Src_IP', 'Dst_IP', 'Timestamp', 'Fwd_PSH_Flags', 'Fwd_URG_Flags', 'Fwd_Byts/b_Avg', 'Fwd_Pkts/b_Avg', 'Fwd_Blk_Rate_Avg', 'Bwd_Byts/b_Avg', 'Bwd_Pkts/b_Avg', 'Bwd_Blk_Rate_Avg', 'Init_Fwd_Win_Byts', 'Fwd_Seg_Size_Min']:
        if col in df.columns:
            df = df.drop(columns=[col])
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)
    df = df.drop_duplicates()
    label_col_guess = 'Cat' if 'Cat' in df.columns else ('Label' if 'Label' in df.columns else 'label')
    labels = df[[label_col_guess]].copy()
    features = df.drop(columns=[c for c in ['Label', 'Cat', 'Sub_Cat'] if c in df.columns], errors='ignore')
    
    train_df, test_df = train_test_split(pd.concat([features, labels], axis=1), test_size=0.2, random_state=100)
    train_df = train_df.rename(columns={label_col_guess: 'label'})
    test_df = test_df.rename(columns={label_col_guess: 'label'})
    
    train_df.to_csv(out_train, index=False)
    test_df.to_csv(out_test, index=False)
    print(f"Đã tạo file train.csv và test.csv tại {data_root}")
    return True

def _download_csv(download_url: str, dest_path: str) -> bool:
    try:
        os.makedirs(os.path.dirname(dest_path), exist_ok=True)
        print(f"Đang tải dữ liệu từ {download_url}...")
        urllib.request.urlretrieve(download_url, dest_path)
        print("Tải xong!")
        return True
    except Exception as e:
        print(f"Tải thất bại: {e}")
        return False

def build_iotid20_loaders(data_root, batch_size=256, num_workers=0, source_csv=None, download_url: str = None):
    train_csv = os.path.join(data_root, 'train.csv')
    test_csv = os.path.join(data_root, 'test.csv')
    if not os.path.exists(train_csv) or not os.path.exists(test_csv):
        ok = _standardize_and_split_source_csv(data_root, train_csv, test_csv, source_csv=source_csv)
        if not ok and download_url:
            downloaded = _download_csv(download_url, os.path.join(data_root, 'IoT_Network_Intrusion_Dataset.csv'))
            if downloaded:
                ok = _standardize_and_split_source_csv(data_root, train_csv, test_csv, source_csv=os.path.join(data_root, 'IoT_Network_Intrusion_Dataset.csv'))
        if not ok:
            raise FileNotFoundError(f'Không tìm thấy file train.csv/test.csv hoặc file CSV nguồn tại {data_root}.')

    label_col = 'label' if 'label' in pd.read_csv(train_csv, nrows=1).columns else 'Cat'
    train_ds = IoTID20CSVDataset(train_csv, label_col=label_col)
    test_ds = IoTID20CSVDataset(test_csv, feature_cols=train_ds.feature_cols, label_col=label_col, scaler=train_ds.scaler, label_encoder=train_ds.label_encoder)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    n_features = train_ds.X.shape[1]
    n_classes = len(train_ds.label_encoder.classes_)
    
    print(f"Tạo Dataloaders thành công: {n_features} features, {n_classes} classes.")
    return train_loader, test_loader, n_features, n_classes

print("Các hàm và lớp xử lý dữ liệu đã sẵn sàng.")

In [ ]:
class quan_Linear(nn.Linear):
    def __init__(self, in_features, out_features, bias=True):
        super(quan_Linear, self).__init__(in_features, out_features, bias=bias)

        self.N_bits = 8
        self.full_lvls = 2**self.N_bits
        self.half_lvls = (self.full_lvls - 2) / 2
        # Initialize the step size
        self.step_size = nn.Parameter(torch.Tensor([1]), requires_grad=True)
        self.__reset_stepsize__()
        # flag to enable the inference with quantized weight or self.weight
        self.inf_with_weight = False  # disabled by default

        # create a vector to identify the weight to each bit
        self.b_w = nn.Parameter(2**torch.arange(start=self.N_bits - 1,
                                                end=-1,
                                                step=-1).unsqueeze(-1).float(),
                                requires_grad=False)

        self.b_w[0] = -self.b_w[0]  #in-place reverse

    def forward(self, input):
        if self.inf_with_weight:
            return F.linear(input, self.weight * self.step_size, self.bias)
        else:
            self.__reset_stepsize__()
            weight_quan = quantize(self.weight, self.step_size,
                                   self.half_lvls) * self.step_size
            return F.linear(input, weight_quan, self.bias)

    def __reset_stepsize__(self):
        with torch.no_grad():
            self.step_size.data = self.weight.abs().max() / self.half_lvls

    def __reset_weight__(self):
        '''
        This function will reconstruct the weight stored in self.weight.
        Replacing the orginal floating-point with the quantized fix-point
        weight representation.
        '''
        # replace the weight with the quantized version
        with torch.no_grad():
            self.weight.data = quantize(self.weight, self.step_size,
                                        self.half_lvls)
        # enable the flag, thus now computation does not invovle weight quantization
        self.inf_with_weight = True


In [ ]:
class quan_Conv1d(nn.Conv1d):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 padding=1,
                 dilation=1,
                 groups=1,
                 bias=True):
        super(quan_Conv1d, self).__init__(in_channels,
                                          out_channels,
                                          kernel_size,
                                          stride=stride,
                                          padding=padding,
                                          dilation=dilation,
                                          groups=groups,
                                          bias=bias)

        # Số lượng bit để lượng tử hóa trọng số
        self.N_bits = 8
        self.full_lvls = 2 ** self.N_bits
        self.half_lvls = (self.full_lvls - 2) / 2

        # Bước lượng tử hóa (step size), là một tham số có thể học được
        self.step_size = nn.Parameter(torch.Tensor([1]), requires_grad=True)
        self.__reset_stepsize__()

        # Cờ để bật hoặc tắt sử dụng trọng số lượng tử hóa
        self.inf_with_weight = False  # Tắt theo mặc định

        # Tạo một vector để biểu diễn trọng số cho từng bit
        self.b_w = nn.Parameter(2 ** torch.arange(start=self.N_bits - 1,
                                                  end=-1,
                                                  step=-1).unsqueeze(-1).float(),
                                requires_grad=False)
        self.b_w[0] = -self.b_w[0]  # Biến đổi MSB thành giá trị âm để hỗ trợ bù hai

    def __reset_stepsize__(self):
        """Hàm này dùng để đặt lại giá trị `step_size`."""
        # Giá trị này có thể được tùy chỉnh tùy thuộc vào yêu cầu của mô hình
        self.step_size.data.fill_(1.0)

    def forward(self, x):
        # Kiểm tra cờ `inf_with_weight` để quyết định sử dụng trọng số đã lượng tử hóa hay không
        if self.inf_with_weight:
            quantized_weight = self.quantize_weight(self.weight)
            return nn.functional.conv1d(x, quantized_weight, self.bias, self.stride,
                                        self.padding, self.dilation, self.groups)
        else:
            return nn.functional.conv1d(x, self.weight, self.bias, self.stride,
                                        self.padding, self.dilation, self.groups)

    def quantize_weight(self, weight):
        """Lượng tử hóa trọng số theo số bit đã định."""
        # Tạo trọng số lượng tử hóa bằng cách sử dụng step_size
        quantized_weight = torch.round(weight / self.step_size) * self.step_size
        quantized_weight = torch.clamp(quantized_weight, -self.half_lvls * self.step_size,
                                       (self.half_lvls - 1) * self.step_size)
        return quantized_weight

In [ ]:
def change_quan_bitwidth(model, n_bit):
    '''This script change the quantization bit-width of entire model to n_bit'''
    for m in model.modules():
        if isinstance(m, quan_Conv1d) or isinstance(m, quan_Linear):
            m.N_bits = n_bit
            # print("Change weight bit-width as {}.".format(m.N_bits))
            m.b_w.data = m.b_w.data[-m.N_bits:]
            m.b_w[0] = -m.b_w[0]
            print(m.b_w)
    return 

In [ ]:

class _quantize_func(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, step_size, half_lvls):
        # ctx is a context object that can be used to stash information
        # for backward computation
        ctx.step_size = step_size
        ctx.half_lvls = half_lvls
        output = F.hardtanh(input,
                            min_val=-ctx.half_lvls * ctx.step_size.item(),
                            max_val=ctx.half_lvls * ctx.step_size.item())

        output = torch.round(output / ctx.step_size)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        grad_input = grad_output.clone() / ctx.step_size

        return grad_input, None, None

quantize = _quantize_func.apply

In [ ]:
class _bin_func(torch.autograd.Function):

    @staticmethod
    def forward(ctx, input, mu):
        ctx.mu = mu
        output = input.clone().zero_()
        output[input.ge(0)] = 1
        output[input.lt(0)] = -1

        return output

    @staticmethod
    def backward(ctx, grad_output):
        grad_input = grad_output.clone() / ctx.mu
        return grad_input, None

w_bin = _bin_func.apply

In [ ]:
class CustomBlock(nn.Module):
    def __init__(self, in_features, out_features, bias=True, apply_softmax=False):
        super(CustomBlock, self).__init__()
        self.N_bits = 16
        self.full_lvls = 2 ** self.N_bits
        self.half_lvls = (self.full_lvls - 2) / 2
        self.apply_softmax = apply_softmax

        # Initialize the step size
        self.step_size = nn.Parameter(torch.Tensor([1]), requires_grad=True)

        # Initialize weights and bias
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        self.bias = nn.Parameter(torch.Tensor(out_features)) if bias else None

        # Reset parameters
        self.__reset_stepsize__()
        self.reset_parameters()

        # Flag for inference with quantized weights
        self.inf_with_weight = False

        self.b_w = nn.Parameter(2**torch.arange(start=self.N_bits - 1,
                                             end=-1,
                                             step=-1).unsqueeze(-1).float(),
                           requires_grad=False)
        self.b_w[0] = -self.b_w[0]  #in-place reverse
        
    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight, a=5 ** 0.5)
        if self.bias is not None:
            nn.init.zeros_(self.bias)

    def forward(self, input):
        if self.inf_with_weight:
            weight_applied = self.weight * self.step_size
        else:
            self.__reset_stepsize__()
            weight_quan = quantize(self.weight, self.step_size, self.half_lvls) * self.step_size
            weight_applied = weight_quan

        # Linear transformation
        input = input.view(input.size(0), -1)  # Flatten input to 2D for matmul
        output = input @ weight_applied.T
        if self.bias is not None:
            output += self.bias

        if self.apply_softmax:
            output = F.softmax(output, dim=-1)
        return output

    def __reset_stepsize__(self):
        with torch.no_grad():
            self.step_size.data = self.weight.abs().max() / self.half_lvls

    def __reset_weight__(self):
        with torch.no_grad():
            self.weight.data = quantize(self.weight, self.step_size, self.half_lvls)
        self.inf_with_weight = True


In [ ]:
class DownsampleA(nn.Module):
    def __init__(self, nIn, nOut, stride):
        super(DownsampleA, self).__init__()
        assert stride == 2
        self.avg = nn.AvgPool1d(kernel_size=1, stride=stride)

    def forward(self, x):
        x = self.avg(x)
        return torch.cat((x, x.mul(0)), 1)
        
class SEBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(SEBlock, self).__init__()
        self.conv_a = quan_Conv1d(inplanes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn_a = nn.BatchNorm1d(planes)
        self.dropout_a = nn.Dropout(p=0.3)  # Dropout sau BatchNorm

        self.conv_b = quan_Conv1d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn_b = nn.BatchNorm1d(planes)
        self.dropout_b = nn.Dropout(p=0.3)  # Dropout sau BatchNorm

        self.downsample = downsample

    def forward(self, x):
        residual = x
        
        basicblock = self.conv_a(x)
        basicblock = self.bn_a(basicblock)
        basicblock = F.relu(basicblock, inplace=True)
        basicblock = self.dropout_a(basicblock)  # Áp dụng dropout

        basicblock = self.conv_b(basicblock)
        basicblock = self.bn_b(basicblock)
        basicblock = self.dropout_b(basicblock)  # Áp dụng dropout

        if self.downsample:
            residual = self.downsample(x)

        return F.relu(residual + basicblock, inplace=True)

In [ ]:
class CustomModel(nn.Module):
    def __init__(self, input_size=69, hidden_sizes=[32, 64, 128, 256, 512], output_size=5):
        super(CustomModel, self).__init__()
        self.fc1 = quan_Conv1d(input_size, hidden_sizes[0], kernel_size=3, stride=1, padding=1)
        self.bn_1 = nn.BatchNorm1d(hidden_sizes[0])

        self.inplanes = 32
        self.stage_1 = self._make_layer(SEBlock, 32, 16, 1)
        self.stage_2 = self._make_layer(SEBlock, 64, 16, 2)
        self.stage_3 = self._make_layer(SEBlock, 128, 16, 2)
        self.avgpool = nn.AdaptiveAvgPool1d(1)

        self.classifier = CustomBlock(128 * SEBlock.expansion, output_size)

        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                n = m.kernel_size[0] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                #m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                init.kaiming_normal(m.weight)
                m.bias.data.zero_()
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        downsample = None
        if stride == 2 or self.inplanes != planes * SEBlock.expansion:
            downsample = DownsampleA(self.inplanes, planes * SEBlock.expansion, stride) if stride == 2 else None

        layers = []
        layers.append(block(self.inplanes, planes, stride=1, downsample=downsample))
        self.inplanes = planes * SEBlock.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(self.bn_1(x), inplace=True)
        
        x = self.stage_1(x)
        x = self.stage_2(x)
        x = self.stage_3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

In [ ]:
class CustomModel2(nn.Module):
    def __init__(self, input_size=69, hidden_sizes=[32, 64, 128, 100], output_size=5):
        super(CustomModel2, self).__init__()
        self.hidden_sizes = hidden_sizes

        # Define layers
        self.fc1 = nn.Conv1d(input_size, hidden_sizes[0], kernel_size=3, stride=2, padding=1)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        
        self.stage_1 = nn.Conv1d(hidden_sizes[0], hidden_sizes[1], kernel_size=3, stride=2, padding=1)
        self.stage_2 = nn.Conv1d(hidden_sizes[1], hidden_sizes[2], kernel_size=3, stride=2, padding=1)
        self.stage_3 = nn.Conv1d(hidden_sizes[2], hidden_sizes[3], kernel_size=3, stride=2, padding=1)

        # Global Pooling
        self.global_pool = nn.AdaptiveAvgPool1d(1)

        # Classifier
        self.classifier = CustomBlock(hidden_sizes[-1], output_size, apply_softmax=True)
        nn.Dropout(0.15)
        
    def forward(self, x):
        # Pass through layers
        x = self.fc1(x)
        x = self.activation(self.pool(x))

        x = self.stage_1(x)
        x = self.activation(self.pool(x))

        x = self.stage_2(x)
        x = self.activation(self.pool(x))

        x = self.stage_3(x)
        x = self.activation(self.pool(x))

        # Global Pooling
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        return self.classifier(x)

In [ ]:
class MLPClassifier(nn.Module):
    def __init__(self, in_features, n_classes):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, n_classes)
        )
    def forward(self, x):
        return self.layers(x)
# Hàm giúp chọn model dựa trên cấu hình
def build_model(model_name, in_features, n_classes):
    if model_name == 'mlp':
        return MLPClassifier(in_features, n_classes)
    elif model_name == 'custom1':
        return CustomModel1(input_size=in_features, output_size=n_classes)
    elif model_name == 'custom2':
        return CustomModel2(input_size=in_features, output_size=n_classes)
    else:
        raise ValueError('Unknown model: ' + model_name)

print("Các kiến trúc model đã sẵn sàng.")

In [ ]:
@torch.no_grad()
def evaluate(model, loader, device='cpu'):
    model.eval()
    total, correct = 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        pred = logits.argmax(1)
        correct += (pred == y).sum().item()
        total += y.size(0)
    return correct / total * 100

def train(model, train_loader, test_loader, epochs, lr, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.to(device)

    for epoch in range(epochs):
        model.train()
        total, correct, loss_sum = 0, 0, 0.0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
            loss_sum += loss.item() * y.size(0)
            pred = logits.argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)
        
        train_acc = correct / total * 100
        test_acc = evaluate(model, test_loader, device)
        print(f'[Epoch {epoch+1:03d}] Train Acc: {train_acc:6.2f}% | Test Acc: {test_acc:6.2f}% | Loss: {loss_sum/total:.4f}')
    
    print("\nHoàn tất huấn luyện!")
    return model

print("Các hàm train/evaluate đã sẵn sàng.")

In [ ]:
class PureCNN(nn.Module):
    """Pure CNN model without quantization for comparison"""
    def __init__(self, input_size=69, output_size=5):
        super().__init__()

        # Convolutional layers with increasing channels
        self.conv1 = nn.Conv1d(input_size, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm1d(64)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm1d(256)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv1d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm1d(512)
        self.pool4 = nn.MaxPool1d(kernel_size=2, stride=2)

        # Adaptive pooling to handle variable input lengths
        self.adaptive_pool = nn.AdaptiveAvgPool1d(1)

        # Fully connected layers
        self.dropout1 = nn.Dropout(0.3)
        self.fc1 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256, 128)
        self.dropout3 = nn.Dropout(0.2)
        self.classifier = nn.Linear(128, output_size)

        # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):
        """Initialize model weights using Xavier initialization"""
        for module in self.modules():
            if isinstance(module, nn.Conv1d):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)
            elif isinstance(module, nn.BatchNorm1d):
                nn.init.ones_(module.weight)
                nn.init.zeros_(module.bias)

    def forward(self, x):
        # Handle input shape: expect [B, 69] or [B, 69, 1]
        if x.dim() == 2:
            x = x.unsqueeze(-1)  # [B, 69] -> [B, 69, 1]

        # If input is [B, 69, 1], transpose to [B, 1, 69] for Conv1d
        if x.size(-1) == 1:
            x = x.transpose(1, 2)  # [B, 69, 1] -> [B, 1, 69]

        # Convolutional blocks
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)

        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)

        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool4(x)

        # Global average pooling
        x = self.adaptive_pool(x)  # [B, 512, 1]
        x = x.view(x.size(0), -1)  # [B, 512]

        # Fully connected layers
        x = self.dropout1(x)
        x = F.relu(self.fc1(x))

        x = self.dropout2(x)
        x = F.relu(self.fc2(x))

        x = self.dropout3(x)
        x = self.classifier(x)
        return x


class EfficientCNN(nn.Module):
    """Lightweight CNN with depthwise separable convolutions"""
    def __init__(self, input_size=69, output_size=5):
        super().__init__()

        # Depthwise separable convolution blocks
        self.conv1 = nn.Conv1d(input_size, input_size, kernel_size=3, stride=1, padding=1, groups=input_size)
        self.pw_conv1 = nn.Conv1d(input_size, 64, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(64)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv1d(64, 64, kernel_size=3, stride=1, padding=1, groups=64)
        self.pw_conv2 = nn.Conv1d(64, 128, kernel_size=1)
        self.bn2 = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv1d(128, 128, kernel_size=3, stride=1, padding=1, groups=128)
        self.pw_conv3 = nn.Conv1d(128, 256, kernel_size=1)
        self.bn3 = nn.BatchNorm1d(256)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)

        # Global pooling and classification
        self.adaptive_pool = nn.AdaptiveAvgPool1d(1)
        self.dropout = nn.Dropout(0.4)
        self.classifier = nn.Linear(256, output_size)

        self._initialize_weights()

    def _initialize_weights(self):
        """Initialize with Xavier uniform"""
        for module in self.modules():
            if isinstance(module, nn.Conv1d):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)

    def forward(self, x):
        # Handle input shape
        if x.dim() == 2:
            x = x.unsqueeze(-1)

        if x.size(-1) == 1:
            x = x.transpose(1, 2)

        # Depthwise separable blocks
        x = F.relu(self.bn1(self.pw_conv1(self.conv1(x))))
        x = self.pool1(x)

        x = F.relu(self.bn2(self.pw_conv2(self.conv2(x))))
        x = self.pool2(x)

        x = F.relu(self.bn3(self.pw_conv3(self.conv3(x))))
        x = self.pool3(x)

        # Global pooling
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)

        x = self.dropout(x)
        return self.classifier(x)


In [ ]:

# 1. Tạo thư mục lưu trữ nếu chưa có
os.makedirs(os.path.dirname(args.OUTPUT_PATH), exist_ok=True)

# 2. Chuẩn bị dữ liệu
train_loader, test_loader, in_features, n_classes = build_iotid20_loaders(
    args.DATA_ROOT, 
    source_csv=args.SOURCE_CSV, 
    download_url=args.DOWNLOAD_URL
)

# 3. Xây dựng model
model = build_model(args.MODEL_NAME, in_features, n_classes)
print(f"Đã xây dựng model '{args.MODEL_NAME}'")
print(model)

# 4. Bắt đầu huấn luyện
trained_model = train(
    model=model,
    train_loader=train_loader, 
    test_loader=test_loader, 
    epochs=args.EPOCHS, 
    lr=args.LEARNING_RATE, 
    device=args.DEVICE
)

# 5. Lưu trọng số của model
torch.save(trained_model.state_dict(), args.OUTPUT_PATH)
print(f'\nĐã lưu trọng số vào file: {args.OUTPUT_PATH}')